This Python code defines and utilizes the `Data_generator` class to generate synthetic data for a physics experiment. The program then cleans up the data by removing outliers and subsequently stores the clean data in a pickle file. 

## Data Generation

`Data_generator` is the heart of this script and is initialized with two parameters: the number of events (`numevents`) and a boolean indicating whether the generated data should be normalized or not. The synthetic data is a representation of the result of physics events involving multiple particles, where each particle is characterized by different properties such as `eta`, `mass`, `phi`, `pt`, `charge`, and `genPartFlav`.

Upon initialization, `Data_generator` configures a series of functions and input variables, which are then utilized in the `generate_fake_data` method to generate synthetic data. 

The `generate_fake_data` method starts by creating a dictionary, `data`, with keys for each variable and empty lists as values. It then generates random data for these variables using relevant statistical distributions which are based on the physical properties being simulated. 

After the data has been generated, `Data_generator` includes an optional step of renaming and reordering keys in the dictionary in a more standard format, before returning the data.

## Data Cleaning

After generating the data, outliers are removed using the `remove_outliers` function. This function uses the limits defined in a YAML file to identify and remove the outliers in the data. 

## Data Storage

Once the outliers have been removed, the data is then stored as a pickle file at a specified location for later use.



In [2]:
import pickle
import numpy as np
from copy import deepcopy
import sys
sys.path.append('../utils/')
from DD_data_extractor_git import Data_generator, remove_all_outliers
import matplotlib.pyplot as plt
import pandas as pd
import yaml
import os

In [2]:
num_events = 100
Data_generator1 = Data_generator(num_events, normalize=True)
data_dict=Data_generator1.getData()

In [3]:
def read_yaml_file(filename):
    with open(filename, 'r') as stream:
        try:
            data = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return data

limits = read_yaml_file('GenerateDataClean.yaml')

def remove_outliers(data, limits):
    outlier_mask = np.zeros(data[next(iter(data))].shape, dtype=bool)  # initially no outliers

    do_not_cut = limits.get('do_not_cut', [])
    cuts = limits.get('cuts', {})

    for feature_name in data.keys():
        if feature_name in do_not_cut:
            continue  # do not change the outlier mask

        feature_limits = cuts.get(feature_name)
        if feature_limits is not None:
            lower_limit, upper_limit = feature_limits.get('lower_percentile', 0.03), feature_limits.get('upper_percentile', 99.7)
        else:
            lower_limit, upper_limit = 0.03, 99.7

        lower_value, upper_value = np.percentile(data[feature_name], [lower_limit, upper_limit])
        feature_outlier_mask = (data[feature_name] < lower_value) | (data[feature_name] > upper_value)
        outlier_mask |= feature_outlier_mask  # update the outlier mask

    # remove outliers from all features
    for feature_name in data.keys():
        data[feature_name] = data[feature_name][~outlier_mask]

    return data



data_dict_removed_outliers2 = deepcopy(data_dict)
data_dict_removed_outliers2 = remove_outliers(data_dict_removed_outliers2, limits)


In [4]:
base_path = "/home/ddemler/dmitri_stuff/"
folder = "saved_models/gnn/raw_data_fake"

full_folder_path = os.path.join(base_path, folder)

os.makedirs(full_folder_path, exist_ok=True)

filename = "data_dict_removed_outliersAug3.pkl"

full_file_path = os.path.join(full_folder_path, filename)

with open(full_file_path, 'wb') as f:
    pickle.dump(data_dict_removed_outliers2, f)

In [5]:
cdpath="/home/ddemler/dmitri_stuff/"

train = pd.read_pickle(cdpath + 'extracted_data/TEST10_v4_train_DD2')
val = pd.read_pickle(cdpath + 'extracted_data/TEST10_v4_val_DD2')
test = pd.read_pickle(cdpath + 'extracted_data/TEST10_v4_test_DD2')
df=pd.concat([train,val,test])